In [ ]:
#| echo: false
#| output: false 
import Pkg; Pkg.activate(".."); Pkg.instantiate()

In [ ]:
#| echo: true
#| code-fold: true
#| code-summary: "Import packages" 
using Revise
using KLLS
using NPZ
using Plots
using Printf
using UnPack
using LinearAlgebra

## Data sets

There are three data sets available:

1. `PhysicsData.npz`
1. `synthetic_data.npz`
1. `Tobias_data.npz`

### Questions

1.  The `Tobias_data.npz` file doesn't seem to be any different from `synthetic_data.npz`. Are these geniunely different data sets?
2. What is the meaning of the `b` vector in `synthetic_data.npz`?
3. How do we normalize the data in `synthetic_data.npz`?

#### Rho-meson


In [ ]:
#| output: true
dataRho = npzread("../data/rho-meson_testproblem.npz", ["A", "x", "b_avg"]) 
probRho = KLLSData(dataRho["A"], dataRho["b_avg"], name="Rho-meson")

Check normalization:

In [ ]:
sum(dataRho["x"])

#### Synthetic UEG


In [ ]:
#| output: true
dataUEG = npzread("../data/synthetic-UEG_testproblem.npz", ["A", "x", "b_avg", "mu"]) 
probUEG = KLLSData(dataUEG["A"], dataUEG["b_avg"], name="UEG")

Check normalization:

In [ ]:
sum(dataUEG["x"])

#### PhysicsData:

In [ ]:
#| output: true
dataPhy = npzread("../data/PhysicsData.npz", ["A", "b", "x0"])
probPhy = KLLSData(dataPhy["A"], dataPhy["b"], name="Physics")

Check normalization:

In [ ]:
sum(dataPhy["x0"])

#### Synthetic data:

For this dataset, the ground truth seems to be named `x` instead of `x0`, as in the other data sets. We'll rename it for consistency:

In [ ]:
#| output: true
dataSyn = npzread("../data/synthetic_data.npz", ["A", "b", "x"])
dataSyn["x0"] = dataSyn["x"] # convenient
probSyn = KLLSData(dataSyn["A"], dataSyn["b"], name="Synthetic Data")

Check normalization:

In [ ]:
sum(dataSyn["x0"])

:::{.callout-important title="Normalization"}
The synthetic data doesn't appear to be normalized. As agreed, we normalize the data such that the first element of the right-hand side vector $b$ is 1:


In [ ]:
#| output: false
probSyn.b ./= probSyn.b[1]
probSyn.A ./= probSyn.b[1]
dataSyn["x0"] ./= probSyn.b[1]

Questions:

1. Note that this normalization still doesn't put `x0` into the simplex:

In [ ]:
sum(dataSyn["x0"])

2. Why is `x0` for this test set stored as single precision float? (See "f" in line above.)
:::

### Visualizing signals

Here are the signals to be recovered:

In [ ]:
#| echo: false
plot(title="Signals to be recovered",
  plot(dataPhy["x0"], label="Physics"),
  plot(dataSyn["x0"], label="Synthetic"),
  plot(dataRho["x"], label="Rho-meson"),
  plot(dataUEG["x"], label="Synthetic UEG"),
)

### Recovery

Recover the signals as the solution of the KL-regularized least squares problem
$$
\min_p \frac{1}{2} \|Ap - b\|^2 + \lambda \sum_{i=1}^n p_i \log(p_i)
$$


In [ ]:
#| output: false
#| code-fold: true
#| code-summary: "Implement test functions"
logrange(start, stop, length) = exp10.(range(start, stop=stop, length=length))
solve_range(prob, λs) = map(λs) do λ
        prob.λ = λ
        p, y, stats = solve!(prob)
        (λ=λ, p=p, iters=stats.iter, solve_time=stats.elapsed_time)
end
function plot_results(stats, x0; title="Recovered distributions")
    lab = hcat([@sprintf("λ = %6.0e", λ) for λ in getfield.(stats, :λ)]...)
    default(lw=2, title=title)
    plot(getfield.(stats, :p), label=lab)
    plot!(x0, label="ground truth")
end

#### Physics data

Solve the problem for a range of logarithmically spaced regularization parameters $\lambda$ between $10^{-16}$ and $10^{-5}$.


In [ ]:
statsPhy = solve_range(probPhy, logrange(-16, -5, 3))
plot_results(statsPhy, dataPhy["x0"])

Report results for smallest value of $\lambda$:

In [ ]:
#| output: false
#| code-fold: true
#| code-summary: "Implement report function" 
function report_errors(stats, data)
  @unpack A, b, x0 = data
  p = stats.p
  m, n = size(A)
  @printf("%20s: %11.2e\n", "rms(p-x0)", norm(p - x0)/√n)
  @printf("%20s: %11.2e\n", "rms(Ap-b)", norm(A*p - b)/√m)
  @printf("%20s: %11.2e\n", "rms(Ax0-b)", norm(A*x0 - b)/√m)
  @printf("%20s: %11f\n", "Solve time (sec)", stats.solve_time)
end

In [ ]:
report_errors(statsPhy[1], dataPhy)

#### Synthetic data

Here we try to recover the solution from the synthetic data set. But as we see here, we get garbage:


In [ ]:
p, y, statsSyn = solve!(probSyn)
plot([p, dataSyn["x0"]], label=["Recovered" "Ground truth"])

:::{.callout-important}
It seems that the vector `b` in that data set isn't correct. What is `b` the meaning of `b` in this data set?
:::

As a test, we construct `b` from the ground truth (forcibly normalized) `x0`:

In [ ]:
let
  @unpack A, x0 = dataSyn
  x0 = x0/sum(x0)
  b = A*x0
  p, y, stats = solve!(KLLSData(A, b))
  plot([p, x0], label=["Recovered" "Ground truth"])
end

### UEG


In [ ]:
let
  probUEG.λ = 1e-6
  p, y, stats = solve!(probUEG)
  plot([p, dataUEG["x"]], label=["Recovered" "Ground truth"])
end